In [1]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd
import re

In [2]:
LIST_PATH = 'https://in-art.ru/art-mir/khudozhniki/'
PREFIX = 'https://in-art.ru'

In [3]:
def get_artist_list(path):    
    page = requests.get(path).content
    soup = bs(page, 'lxml')
    for script in soup.find_all('script'):
        if script.text.find('artistsData') != -1:
            data = script.text
            data = data.replace('null', '"null"')
            data = data[data.find('{'):data.find(r'}};') + 2]
            parsed_artists = json.loads(data, encoding='utf8')['ITEMS']
            break
    for artist in parsed_artists:
        del artist['PICTURES']

    return parsed_artists

In [5]:
def get_artist_data(url):
    page = requests.get(url).content
    soup = bs(page, 'lxml')
    tech = soup.find('div', text=re.compile(r'\s*Техника\s*'))
    if tech is not None:
        tech = tech.parent.find_all('div')[1].text.strip()
    personal_exh = soup.find(text=re.compile(r'\s*Персональные выставки\s*'))
    if personal_exh is not None:
        personal_exh = personal_exh.parent.parent.find_all('div')[2].find_all('div')[0].text
    collective_exh = soup.find(text=re.compile(r'\s*Групповые выставки\s*'))
    if collective_exh is not None:
        collective_exh = collective_exh.parent.parent.find_all('div')[3].text
    return tech, personal_exh, collective_exh

In [6]:
artists_list = get_artist_list(LIST_PATH)
artists_data = []

In [7]:
for i in range(0, len(artists_list)):
    artist = artists_list[i]
    print(artist['NAME'])
    tech, personal_exh, collective_exh = get_artist_data(PREFIX + artist['DETAIL_PAGE_URL'])
    name = artist['NAME']
    artists_data.append({'name': name, 'style': tech, 'personal_exh': personal_exh, 'collective_exh': collective_exh})

Crocodile POWER
MIXAM  MAXIM
Megasoma Mars
Moff Вячеслав
Rave Kirill
Roiter Andrei
Street Udmurt
ZIP
Абалакова Наталья
Абрамишвили Гия
Абрамов Андрей
Аввакумов  Юрий
Агроскин Семен
Агуреева  Мария
Аджер (Кузнецов) Константин
Акрамов Сергей
Аксенов Петр
Аксинин Александр
Аксёнов Павел
Акулин Даня
Алдошин Владимир
Александров Владимир (Император Вава)
Александров Кирилл
Александров Юрий
Алексеев Дмитрий
Алексеев Никита
Алексеев Николай
Алексеева Марина
Алимпиев Виктор
Алпатов Алексей
Алфёров Сергей
Альберт  Юрий
Алёша
Анзельм Владимир
Аношенкова Людмила
Анро Андрей
Антошина Татьяна
Антуфьев Евгений
Ануфриев  Сергей
Анфалова Надежда
Апухтина Валентина
Арендт Мария
Артамонова Лиза
Архипенко  Даниил
Архипов  Владимир
Аске  Дмитрий
Асс Евгений
Ахломов Виктор
Ахметгалиева  Татьяна
Бабич Юрий
Багс Стас
Бадалов Баби
Баданина Татьяна
Баевер Антонина
Базилев Сергей
Бакин Сергей
Баловин  Сергей
Баров  Андрей
Бартенев Андрей
Барыкин Валерий
Басалаев  Кирилл
Басов  Михаил
Басыров Гариф
Батраков Илья

Острецов Гоша
Отдельнов  Павел
Панкин Александр
Панфилова Дарья
Паперно Александра
Паркина Анна
Парфенюк Сергей
Парщиков  Тимофей
Патракеев Денис
Пахомов Сергей
Пепперштейн Павел
Первов Георгий
Петракова Ирина
Петрелли Александр
Петров Аркадий
Пивоваров Виктор
Пинхасов Георгий
Пирогова Саша
Плавинский Дмитрий
Плотников  Егор
Плюснин Александр
Плющ Иван
Повзнер Александр
Повзнер Лев
Погоржельская Мария
Погоржельский  Александр
Подмаркова Татьяна
Подобед Саша
Полисский Николай
Политов & Белова
Польский Антон MAKE
Поляков  Данила
Поляков Константин
Пономарев Олег
Пономарёв Святослав
Пономарёва Вероника
Попова Елена
Потапов  Владимир
Пригов Дмитрий
Провоторов Дмитрий
Птрк (Комиссаров) Слава
Пузенков Георгий
Пурыгин Леонид
Пушницкий Виталий
Пёникер  Таня
Рабин Оскар
Рагимов Керим
Рагозина Марина
Радя  Тимофей
Разумов Иван
Ракузин Нафтали
Рачуйко  Пасмур
Ребус Дима
Рейхет Петр
Рибас Виктор
Рогинский Михаил
Родченко Александр
Рожин  Сергей
Розанов Михаил
Розенбаум Наталия
Ройтбурд Александр
Р

In [8]:
artists_df = pd.DataFrame(artists_data)
artists_df.to_csv('artists_data.csv')

In [9]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(artists_data, f)